In [1]:
import numpy as np
from scipy.linalg import svd

# **st-HOSVD**

In [4]:
def sthosvd(T, ranks = None, eps = 10**(-8)):
  d = T.ndim
  G = T.copy()
  U = []
  rel_eps = eps * np.linalg.norm(T)**2

  if ranks == None:
    ranks = np.zeros(d, dtype= int)

  for k in range(d):
    T_k = np.moveaxis(G, k, 0).reshape(G.shape[k], -1)
    U_k, sigmas_k, V_k = svd(T_k)

    if ranks[k] == 0:
      ranks[k] = sigmas_k.shape[0]
      s = sigmas_k[-1]**2;
      while s < rel_eps:
        ranks[k] -= 1
        s += sigmas_k[-(sigmas_k.shape[0] - ranks[k])]**2
      ranks[k] += 1

    U_trunc = U_k[:, :ranks[k]]
    U.append(U_trunc)

    G = np.moveaxis(np.tensordot(U_trunc, G, axes=([0], [k])), 0, k)
  return G, U, ranks

In [5]:
shape = (20, 30, 40)
i, j, k = np.indices(shape)
T = np.sin(i + j + k)

G, U, ranks = sthosvd(T)

for k in range(G.ndim):
  G = np.moveaxis(np.tensordot(U[k], G, axes=(1, k)), 0, k)

print(np.linalg.norm(T - G)**2 / np.linalg.norm(T)**2)

9.061340163742612e-31
